In [1]:
from requests import get
import bs4
from bs4 import BeautifulSoup


import pandas as pd
import numpy as np

from time import sleep
from random import randint

from time import time
from warnings import warn

from datetime import datetime
from datetime import timedelta
from pytz import timezone

from IPython.display import clear_output


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

import re
import os
import sys

import glob
import shutil
from pathlib import Path

In [2]:
# Path of the data folder
folder_data = r'C:\Users\motodangiuuuu\Crawler'

# Settings for web driver and Proxy
chrome_driver_path = os.path.join(os.getcwd(),'chromedriver.exe')
# PROXY = "socks5://localhost:9050" # IP:PORT or HOST:PORT

# create a new Chrome session
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
# options.add_argument('--headless')
# options.add_argument('--proxy-server=%s' % PROXY)
driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)
driver.implicitly_wait(0)

## Lấy link các công việc

In [3]:
list_urls_main = ['https://careerbuilder.vn/viec-lam/th%C6%B0%C6%A1ng-m%E1%BA%A1i-%C4%91i%E1%BB%87n-t%E1%BB%AD-k-vi.html']
for i in range(1, 5):
    url = 'https://careerbuilder.vn/viec-lam/th%C6%B0%C6%A1ng-m%E1%BA%A1i-%C4%91i%E1%BB%87n-t%E1%BB%AD-k-trang-' + str(i) +'-vi.html'
    list_urls_main.append(url)
list_urls_main

['https://careerbuilder.vn/viec-lam/th%C6%B0%C6%A1ng-m%E1%BA%A1i-%C4%91i%E1%BB%87n-t%E1%BB%AD-k-vi.html',
 'https://careerbuilder.vn/viec-lam/th%C6%B0%C6%A1ng-m%E1%BA%A1i-%C4%91i%E1%BB%87n-t%E1%BB%AD-k-trang-1-vi.html',
 'https://careerbuilder.vn/viec-lam/th%C6%B0%C6%A1ng-m%E1%BA%A1i-%C4%91i%E1%BB%87n-t%E1%BB%AD-k-trang-2-vi.html',
 'https://careerbuilder.vn/viec-lam/th%C6%B0%C6%A1ng-m%E1%BA%A1i-%C4%91i%E1%BB%87n-t%E1%BB%AD-k-trang-3-vi.html',
 'https://careerbuilder.vn/viec-lam/th%C6%B0%C6%A1ng-m%E1%BA%A1i-%C4%91i%E1%BB%87n-t%E1%BB%AD-k-trang-4-vi.html']

In [4]:
job={}
list_urls = []
for link in list_urls_main:
    driver.get(link)
    sleep(3)
    
    element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="jobs-side-list"]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup_level1=BeautifulSoup(html_of_interest, 'lxml') 
    
    job_title = soup_level1.select('a[class="job_link"]')
 ## List chi tiet cac cong viec
    for i in job_title:
        job[i.text.strip()] = i['href']
        list_urls.append(i['href'])

In [7]:
list_urls[:10]

250

In [5]:
url = 'https://careerbuilder.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh-kenh-thuong-mai-dien-tu.35B5EE16.html'
driver.get(url)

In [6]:
element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="template template01 "]')))
html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
soup_level1=BeautifulSoup(html_of_interest, 'lxml')
print("Block 3:", url)

Block 3: https://careerbuilder.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh-kenh-thuong-mai-dien-tu.35B5EE16.html


In [14]:
dict_detail = {}   

get_title_job(soup_level1, dict_detail)

{'Tên việc làm': 'Nhân viên kinh doanh kênh thương mại điện tử'}

In [15]:
dict_detail['Link'] = url
get_welfares_info(soup_level1, dict_detail)

{'Tên việc làm': 'Nhân viên kinh doanh kênh thương mại điện tử',
 'Link': 'https://careerbuilder.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh-kenh-thuong-mai-dien-tu.35B5EE16.html',
 'Phúc lợi': ['Chế độ bảo hiểm',
  'Phụ cấp',
  'Chế độ thưởng',
  'Đào tạo',
  'Tăng lương',
  'Công tác phí',
  'Nghỉ phép năm']}

In [16]:
dict_detail

{'Tên việc làm': 'Nhân viên kinh doanh kênh thương mại điện tử',
 'Link': 'https://careerbuilder.vn/vi/tim-viec-lam/nhan-vien-kinh-doanh-kenh-thuong-mai-dien-tu.35B5EE16.html',
 'Phúc lợi': ['Chế độ bảo hiểm',
  'Phụ cấp',
  'Chế độ thưởng',
  'Đào tạo',
  'Tăng lương',
  'Công tác phí',
  'Nghỉ phép năm']}

In [13]:
def get_title_job(soup_level1, dict_detail):
    title_job = soup_level1.select('div[class="head-left"]')[0].select('div[class="title"] > h2')
    dict_detail['Tên việc làm'] = title_job[0].text
    return dict_detail

#Hàm lấy thông tin Phúc lợi của công ty
def get_welfares_info(soup_level1, dict_detail):
    welfares = soup_level1.select('div[class="detail-row box-welfares"]')[0].select('li')
    phuc_loi=[]
    for i in welfares:
        phuc_loi.append(i.text.strip())
        dict_detail['Phúc lợi'] = phuc_loi
    return dict_detail

In [ ]:
def detail_template_1(soup_level1): 
    
    def get_title_job(soup_level1, dict_detail):
    title_job = soup_level1.select('div[class="head-left"]')[0].select('div[class="title"] > h2')
    dict_detail['Tên việc làm'] = title_job[0].text
    return dict_detail

    #Hàm lấy thông tin Phúc lợi của công ty
    def get_welfares_info(soup_level1, dict_detail):
        welfares = soup_level1.select('div[class="detail-row box-welfares"]')[0].select('li')
        phuc_loi=[]
        for i in welfares:
            phuc_loi.append(i.text.strip())
            dict_detail['Phúc lợi'] = phuc_loi
        return dict_detail    
    
    def get_summary_info(soup_level1, dict_detail):
        map_div = soup_level1.select('div[class="box-right"]')[0].select('td')
        i=0
        while i < len(map_div):
            dict_detail[map_div[i].text.strip()] = map_div[i+1].text.strip()
            i += 2
        return dict_detail 
    
    #Hàm lấy thông tin Mô tả công việc, yêu cầu và thông tin khác của công ty => chỉ dùng block 3, 4, 5
    def get_job_desc(soup_level1, dict_detail):
        map_div = soup_level1.select('div[class="detail-row"]')

        for item in range(len(map_div)):
            attribute = map_div[item]
            att =""
            for item in attribute.select('p, ul'):
                att = att + str(item) +("\n")
                att = att.replace("<", "[").replace(">", "]")
            dict_detail[attribute.select('h3')[0].text.strip().capitalize()] = att
        return dict_detail
    
    #Hàm lấy thông tin Phúc lợi của công ty => chỉ dùng block 3, 4, 5
    def get_job_tags(soup_level1, dict_detail):
        job_tags = soup_level1.select('div[class="job-tags detail-row"]')[0]
        jobtags_skills=[]
        tag_skills = job_tags.select('li')
        if tag_skills:
            for tag in tag_skills:
                jobtags_skills.append(tag.text)
        dict_detail['Job tags / Skills'] = jobtags_skills  

        return dict_detail
    


    dict_detail = {}
    dict_detail['Link'] = url

    get_title_job(soup_level1, dict_detail)    
    get_welfares_info(soup_level1, dict_detail)
    get_summary_info(soup_level1, dict_detail)
    get_job_desc(soup_level1, dict_detail)
    get_job_tags(soup_level1, dict_detail)


        
    return dict_detail

In [ ]:
list_items = []
try: ##Block3
    element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="template template01 "]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup_level1=BeautifulSoup(html_of_interest, 'lxml')
    print("Block 3:", url)
    
    dict_detail = detail_template_1(soup_level1)
    list_items.append(dict_detail)
    
except TimeoutException:
    try:  ##Block2
        element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="template template01 "]')))
        html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
        soup_level1=BeautifulSoup(html_of_interest, 'lxml')
        print("Block 3:", url)

        dict_detail = detail_template_1(soup_level1)
        list_items.append(dict_detail)
        
    except TimeoutException:
        try:  ##Block4
            element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="template template01 "]')))
            html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
            soup_level1=BeautifulSoup(html_of_interest, 'lxml')
            print("Block 3:", url)

            dict_detail = detail_template_1(soup_level1)
            list_items.append(dict_detail)
    





In [7]:
def detail_template_2(soup_level1, dict_detail)
    #Hàm lấy thông tin Phúc lợi của công ty
    def get_welfares_info(soup_level1, dict_detail):
        welfares = soup_level1.select('div[class="detail-row box-welfares"]')[0].select('li')
        phuc_loi=[]
        for i in welfares:
            phuc_loi.append(i.text.strip())
            dict_detail['Phúc lợi'] = phuc_loi
        return dict_detail

In [8]:
#Hàm lấy thông tin Phúc lợi của công ty => chỉ dùng template 1, 2
def get_job_tags_fs(soup_level1, dict_detail):     
    jobtags_skills=[]
    tag_skills = soup_level1.select('div[class="job-tags"] > ul > li')
    if tag_skills:
        for tag in tag_skills:
            jobtags_skills.append(tag.text)
    dict_detail['Job tags / skills'] = jobtags_skills
    return dict_detail

In [9]:
#Hàm lấy thông tin Phúc lợi của công ty => chỉ dùng block 3, 4, 5
def get_job_tags(soup_level1, dict_detail):
    job_tags = soup_level1.select('div[class="job-tags detail-row"]')[0]
    jobtags_skills=[]
    tag_skills = job_tags.select('li')
    if tag_skills:
        for tag in tag_skills:
            jobtags_skills.append(tag.text)
    dict_detail['Job tags / Skills'] = jobtags_skills  

    return dict_detail

In [10]:
#Hàm lấy thông tin Mô tả công việc, yêu cầu và thông tin khác của công ty => chỉ dùng block 3, 4, 5
def get_job_desc(soup_level1, dict_detail):
    map_div = soup_level1.select('div[class="detail-row"]')

    for item in range(len(map_div)):
        attribute = map_div[item]
        att =""
        for item in attribute.select('p, ul'):
            att = att + str(item) +("\n")
            att = att.replace("<", "[").replace(">", "]")
        dict_detail[attribute.select('h3')[0].text.strip().capitalize()] = att
    return dict_detail

In [11]:
#Hàm lấy thông tin Mô tả công việc, yêu cầu và thông tin khác của công ty => chỉ dùng block 3, 4, 5
def get_job_detail(soup_level1, class_name, dict_detail):
    map_div = soup_level1.select('div[class=class_name]')[0].select('td')
    i=0
    while i < len(map_div):
        dict_detail[map_div[i].text.strip()] = map_div[i+1].text.strip()
        i += 2
    return dict_detail

In [12]:
#Hàm lấy thông tin Mô tả công việc, yêu cầu và thông tin khác của công ty => chỉ dùng block 3, 4, 5
def get_title_job(soup_level1, dict_detail):
    title_job = soup_level1.select('div[class="head-left"]')[0].select('div[class="title"] > h2')
    dict_detail['Tên việc làm'] = title_job[0].text
    return dict_detail

In [ ]:
try:  ##Block1
    soup_level1
    detail_template_1
except :
    try
        soup_level1
        detail_template_1
        


In [21]:
list_items = []
for url in list_urls[250:300]:
    driver.get(url)
    sleep(3)
    dict_detail = {}
    
    try:  ##Block1
        element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="search-result-list-detail template-2"]')))
        html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
        soup_level1=BeautifulSoup(html_of_interest, 'lxml')
        print("Block 1:", url)

        #Tên việc làm
        map_div = soup_level1.select('div[class="apply-now-content"] > div[class="job-desc"] > h1')
        if map_div[0].text:
            dict_detail['Tên việc làm'] = map_div[0].text   

        #Link   
        dict_detail['Link'] = url

        #Địa điểm   
        map_div = soup_level1.select('div[class="detail-box"] > div[class="map"]')
        dict_detail['Nơi làm việc'] = map_div[0].select('p > a')[0].text

        #Ngày cập nhật   #Ngành nghề  #Hình thức    #Lương, Kinh nghiệm, Hết hạn nộp, Cấp bậc
        map_div = soup_level1.select('div[class="col-lg-4 col-sm-6 item-blue"] > div[class="detail-box has-background"]')
        for i in range(0, 2):
            for item in range(0, len(map_div[i].select('ul > li'))):
                dict_detail[map_div[i].select('ul > li')[item].select('li > strong')[0].text.strip()] = map_div[i].select('ul > li')[item].select('p')[0].text.strip()

        #Phúc lợi    
        for i in range(len(soup_level1.select('div[class="detail-row"]'))):
            attribute = soup_level1.select('div[class="detail-row"]')[i]
            if attribute.select('h3'):
                if attribute.select('h3')[0].text.strip() == 'Phúc lợi':
                    phuc_loi=[]
                    for i in attribute.select('ul> li'):
                        phuc_loi.append(i.text.strip())
                        dict_detail[attribute.select('h3')[0].text.strip()] = phuc_loi
                elif attribute.select('h3')[0].text.strip() == 'Thông tin khác':
                    tt_khac=[]
                    for i in attribute.select('ul> li'):
                        tt_khac.append(i.text.strip())
                        dict_detail[attribute.select('h3')[0].text.strip()] = tt_khac
                else:
                    att =""
                    for item in attribute.select('p, ul'):
                        att = att + str(item) +("\n")
                    att = att.replace("<", "[").replace(">", "]")
                    dict_detail[attribute.select('h3')[0].text.strip()] = att

        #Job Tags/Skills       
        get_job_tags_fs(soup_level1, dict_detail)

        # #Tổng quan công ty
#         link_company_info = soup_level1.select('a[class="employer job-company-name"]')[0]['href']
#         get_company_info(link_company_info, dict_detail)
            
        list_items.append(dict_detail)
    
    
    except TimeoutException:
        try:  ##Block2
            element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="search-result-list-detail"]')))
            html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
            soup_level1=BeautifulSoup(html_of_interest, 'lxml')  
            print("Block 2:", url)

            #Tên việc làm
            map_div = soup_level1.select('div[class="apply-now-content"] > div[class="job-desc"] > h1')
            if map_div[0].text:
                dict_detail['Tên việc làm'] = map_div[0].text   
                
            #Link   
            dict_detail['Link'] = url
            
            #Địa điểm   #Ngày cập nhật   #Ngành nghề  #Hình thức    #Lương, Kinh nghiệm, Hết hạn nộp, Cấp bậc
            map_div = soup_level1.select('div[id="info-career-desktop"] > ul >li')
            if map_div:
                for item in range(len(map_div)):
                    dict_detail[map_div[item].select('li > b')[0].text] = map_div[item].select('div')[0].text.strip()

            #Phúc lợi    #Mô tả, yêu cầu công việc, thông tin khác
            for i in range(len(soup_level1.select('div[class="detail-row"]'))):
                attribute = soup_level1.select('div[class="detail-row"]')[i]
                if attribute.select('h3'):
                    if attribute.select('h3')[0].text.strip() == 'Phúc lợi':
                        phuc_loi=[]
                        for i in attribute.select('ul> li'):
                            phuc_loi.append(i.text.strip())
                            dict_detail[attribute.select('h3')[0].text.strip()] = phuc_loi
                    elif attribute.select('h3')[0].text.strip() == 'Thông tin khác':
                        tt_khac=[]
                        for i in attribute.select('ul> li'):
                            tt_khac.append(i.text.strip())
                            dict_detail[attribute.select('h3')[0].text.strip()] = tt_khac
                    else:
                        att =""
                        for item in attribute.select('p, ul'):
                            att = att + str(item) +("\n")
                        att = att.replace("<", "[").replace(">", "]")
                        dict_detail[attribute.select('h3')[0].text.strip()] = att
                        
            #Job Tags/Skills       
            get_job_tags_fs(soup_level1, dict_detail)    
                
#             link_company_info = soup_level1.select('td >a')[0]['href']
#             get_company_info(link_company_info, dict_detail)
            
            list_items.append(dict_detail)
        
        
        except TimeoutException:
            try: ##Block3
                element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="template template01 "]')))
                html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
                soup_level1=BeautifulSoup(html_of_interest, 'lxml')
                print("Block 3:", url)

                get_title_job(soup_level1, dict_detail)

                dict_detail['Link'] = url

                get_welfares_info(soup_level1, dict_detail)

#                 get_job_detail(soup_level1, , dict_detail)
                map_div = soup_level1.select('div[class="box-right"]')[0].select('td')
                i=0
                while i < len(map_div):
                    dict_detail[map_div[i].text.strip()] = map_div[i+1].text.strip()
                    i += 2
                
                get_job_desc(soup_level1, dict_detail)

                get_job_tags(soup_level1, dict_detail)
                
#                 link_company_info = soup_level1.select('a[class="company"]')[0]['href']
#                 get_company_info(link_company_info, dict_detail)
                
                list_items.append(dict_detail)
                
            except TimeoutException:
                try: ##Block4
                    element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="template template02 "]')))
                    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
                    soup_level1=BeautifulSoup(html_of_interest, 'lxml')
                    print("Block 4:", url)

                    get_title_job(soup_level1, dict_detail)

                    dict_detail['Link'] = url

                    map_div = soup_level1.select('div[class="box-info"]')[0].select('td')
                    i=0
                    while i < len(map_div):
                        dict_detail[map_div[i].text.strip()] = map_div[i+1].text.strip()
                        i += 2

                    get_job_desc(soup_level1, dict_detail)
                    
                    get_welfares_info(soup_level1, dict_detail)
                            
                    get_job_tags(soup_level1, dict_detail) 
                
#                     link_company_info = soup_level1.select('a[class="company"]')[0]['href']
#                     get_company_info(link_company_info, dict_detail)

                    list_items.append(dict_detail)
                    
                except TimeoutException: ##Block5
                    element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="template template04 "]')))
                    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
                    soup_level1=BeautifulSoup(html_of_interest, 'lxml')
                    print("Block 5:", url)
                    
                    get_title_job(soup_level1, dict_detail)

                    dict_detail['Link'] = url

                    #Thông tin tuyển dụng
#                     get_job_detail(soup_level1, "box-info", dict_detail)
                    map_div = soup_level1.select('div[class="box-info"]')[0].select('td')
                    i=0
                    while i < len(map_div):
                        dict_detail[map_div[i].text.strip()] = map_div[i+1].text.strip()
                        i += 2                    
                    get_welfares_info(soup_level1, dict_detail)

                    map_div = soup_level1.select('div[class="full-content"]')[0].select('div[class="detail-row"]')

                    #Mô tả công việc
                    att =""
                    for item in map_div[0].select('p, ul'):
                        att = att + str(item) +("\n")
                        att = att.replace("<", "[").replace(">", "]")
                    dict_detail["Mô tả Công việc"] = att

                    for item in range(1, len(map_div)):
                        attribute = map_div[item]  
                        att =""
                        for item in attribute.select('p, ul'):
                            att = att + str(item) +("\n")
                            att = att.replace("<", "[").replace(">", "]")
                        dict_detail[attribute.select('h3')[0].text.strip().capitalize()] = att

                    get_job_tags(soup_level1, dict_detail)  
                    
#                     link_company_info = soup_level1.select('a[class="company"]')[0]['href']
#                     get_company_info(link_company_info, dict_detail)
            
                    list_items.append(dict_detail)

# list_items


In [9]:
df = pd.DataFrame(list_items)

In [9]:
path_cralwer = r'C:\Users\motodangiuuuu\Crawler\Data'
saumuoi.to_csv(os.path.join(path_cralwer, "0_to_60_jobs.csv"), index=False)

In [11]:
df = pd.DataFrame.from_dict(dict_detail, orient='index')
df = df.transpose()
df

In [5]:
# # nam_muoi
# nam_muoi.rename(columns={'Địa điểm':'Nơi làm việc',
#                           'Mô tả Công việc':'Mô tả công việc',
#                           'Yêu Cầu Công Việc':'Yêu cầu công việc'}, 
#                  inplace=True)
# namuoi = pd.read_csv(os.path.join(path_cralwer, "0_to_150_jobs.csv"))
# list_items

In [200]:
url = 'https://careerbuilder.vn/vi/tim-viec-lam/gap-nhan-vien-sale-online-thuong-mai-dien-tu.35B5B7A0.html'
driver.get(url)

In [225]:
url = 'https://careerbuilder.vn/vi/tim-viec-lam/truong-phong-kinh-doanh-thuong-mai-dien-tu-thu-nhap-tu-25-40-trieu.35B55FA6.html'
driver.get(url)

In [226]:
try:
    element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="search-result-list-detail template-2"]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup_level1=BeautifulSoup(html_of_interest, 'lxml')
except TimeoutException:
    element= WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'section[class="search-result-list-detail"]')))
    html_of_interest=driver.execute_script('return arguments[0].innerHTML',element)
    soup_level1=BeautifulSoup(html_of_interest, 'lxml')
    

In [17]:
# soup_level1

In [227]:
dict_detail = {}


## Tên việc làm

In [150]:
map_div = soup_level1.select('div[class="apply-now-content"] > div[class="job-desc"] > h1')
# dict_detail['Tên việc làm'] = map_div[0].text
# dict_detail['Tên việc làm']

[<h1 class="title">Chuyên viên Thương mại Điện tử</h1>]

In [151]:
#v2
map_div = soup_level1.select('div[class="apply-now-content"] > div[class="job-desc"] > h1')
dict_detail['Tên việc làm'] = map_div[0].text
dict_detail['Tên việc làm']

'Chuyên viên Thương mại Điện tử'

# Chi tiết công việc

In [187]:
#v2
map_div = soup_level1.select('div[id="info-career-desktop"] > ul >li')
for item in range(len(map_div)):
    dict_detail[map_div[item].select('li > b')[0].text] = map_div[item].select('div')[0].text.strip()
dict_detail

{'Nơi làm việc': 'Hồ Chí Minh',
 'Cấp bậc': 'Nhân viên',
 'Lương': '12,000,000 - 15,000,000 VND',
 'Hết hạn nộp': '11/03/2021',
 'Ngành nghề': 'Tiếp thị / Marketing, Bán hàng / Kinh doanh, Tiếp thị trực tuyến',
 'Kinh nghiệm': '2 - 3 Năm',
 'Địa điểm': ''}

## Địa điểm

In [179]:
map_div = soup_level1.select('div[class="detail-box"] > div[class="map"]')
dict_detail['Địa điểm'] = map_div[0].select('p > a')[0].text
dict_detail['Địa điểm']

IndexError: list index out of range

## Ngày cập nhật - Hết hạn nộp

In [49]:
map_div = soup_level1.select('div[class="col-lg-4 col-sm-6 item-blue"] > div[class="detail-box has-background"]')
# if map_div:
#     print(map_div)

## Ngày cập nhật

In [50]:
dict_detail['Ngày cập nhật'] = map_div[0].select('ul > li')[0].select('p')[0].text
dict_detail['Ngày cập nhật']

'18/03/2021'

## Ngành nghề

In [186]:
nganh_nghe =[]
for i in map_div[0].select('ul > li')[1].select('a'):
#     print(i)
    nganh_nghe.append(i.text.strip())
dict_detail['Ngành nghề'] = nganh_nghe
dict_detail['Ngành nghề']

IndexError: list index out of range

## Hình thức

In [97]:
dict_detail['Hình thức'] = map_div[0].select('ul > li')[2].select('p')[0].text
dict_detail['Hình thức']

' Hình thức'

## Lương

In [53]:
dict_detail['Lương'] = map_div[1].select('ul > li')[0].select('p')[0].text
dict_detail['Lương']

'Cạnh tranh'

## Kinh nghiệm

In [54]:
dict_detail['Kinh nghiệm'] = map_div[1].select('ul > li')[1].select('p')[0].text.strip()
dict_detail['Kinh nghiệm']

'2 -  Năm'

## Cấp bậc

In [55]:
dict_detail['Cấp bậc'] = map_div[1].select('ul > li')[2].select('p')[0].text.strip()
dict_detail['Cấp bậc']

'Nhân viên'

## Hết hạn nộp

In [56]:
dict_detail['Hết hạn nộp'] = map_div[1].select('ul > li')[3].select('p')[0].text.strip()
dict_detail['Hết hạn nộp']
dict_detail

{'Tên việc làm': 'Nhân Viên Kinh Doanh Thương Mại Điện Tử/ Sales Ecommerce',
 'Địa điểm': 'Hồ Chí Minh',
 'Ngày cập nhật': '18/03/2021',
 'Ngành nghề': ['Bán hàng / Kinh doanh', 'Điện / Điện tử / Điện lạnh'],
 'Hình thức': 'Nhân viên chính thức',
 'Lương': 'Cạnh tranh',
 'Kinh nghiệm': '2 -  Năm',
 'Cấp bậc': 'Nhân viên',
 'Hết hạn nộp': '11/04/2021'}

# Phúc lợi

In [199]:
welfare_list = soup_level1.select('div[class="detail-row"]')[0]
welfare_list_r = welfare_list.select('h3')[0].text
# if welfare_list:
#     print(welfare_list[0])
welfare_list.select('p, ul')

[<ul class="welfare-list">
 <li><span class="fa fa-laptop"></span> Laptop</li>
 <li><span class="fa fa-medkit"></span> Chế độ bảo hiểm</li>
 <li><span class="fa fa-usd"></span> Chế độ thưởng</li>
 <li><span class="fa fa-graduation-cap"></span> Đào tạo</li>
 <li><span class="fa fa-line-chart"></span> Tăng lương</li>
 <li><span class="fa fa-briefcase"></span> Nghỉ phép năm</li>
 </ul>]

In [173]:
phuc_loi =[]
for i in welfare_list[0].select('li'):
    phuc_loi.append(i.text.strip())
dict_detail['Phúc lợi'] = phuc_loi
dict_detail['Phúc lợi']   

['Laptop',
 'Chế độ bảo hiểm',
 'Du Lịch',
 'Chế độ thưởng',
 'Chăm sóc sức khỏe',
 'Đào tạo',
 'Tăng lương',
 'Công tác phí',
 'Nghỉ phép năm',
 'CLB thể thao']

# Mô tả công việc

In [174]:
job_desc = soup_level1.select('div[class="detail-row"]')[1]

job_describe =""
for item in job_desc.select('p, ul'):
    job_describe = job_describe + str(item) +("\n")
job_describe = job_describe.replace("<", "[").replace(">", "]")
dict_detail['Mô tả công việc'] = job_describe
# dict_detail['Mô tả công việc']  

# Yêu cầu công việc

In [175]:
job_require = soup_level1.select('div[class="detail-row"]')[2]

job_requirements =""
for item in job_require.select('p, ul'):
    job_requirements = job_requirements + str(item) +("\n")
job_requirements = job_requirements.replace("<", "[").replace(">", "]")
dict_detail['Yêu cầu công việc'] = job_requirements
dict_detail['Yêu cầu công việc']  

'[ul][li]Tốt nghiệp Cao đẳng trở lên khối ngành kinh tế, thương mại, marketing hoặc liên quan.[/li][li]Kinh nghiệm 2 - 3\xa0năm trên kênh TMĐT, ưu tiên có kinh nghiệm triển khai các sản phẩm mới.[/li][li]Có hiểu biết về SEO / E-commerce / Web Design / Social Media / Copywriting\xa0[/li][li]Sử dụng thành thạo Microsoft office, biết các phần mềm thiết kế, có kiến thức tốt về internet và các xu hướng online marketing[/li][li]Giao tiếp tốt\xa0bằng Tiếng Anh[/li][li]Khả năng sáng tạo tốt[/li][li]Trách nhiệm, kỷ luật, nhiệt tình[/li][/ul]\n[p]THÔNG TIN KHÁC[/p]\n[ul][li][p]Cơ hội huấn luyện:[br/]- Huấn luyện sản phẩm, quy trình công việc trong nội bộ.[br/]- Tham gia đào tạo về kỹ năng công việc, kỹ năng mềm trong nội bộ hoặc bởi các trung tâm đào tạo chuyên nghiệp, đối tác nước ngoài.[br/]- Tham gia các chương trình phát triển năng lực để phát triển sự nghiệp trong hệ thống công ty.[/p][/li][li][p]Đồng nghiệp:[br/]Môi trường trẻ năng động , gắn kết cao, hòa đồng. Đề cao sức sáng tạo, phát tr

In [ ]:
## xử lý \xa0 \xa0

# Thông tin khác

In [176]:
tt_khac = soup_level1.select('div[class="detail-row"]')[3]
# tt_khac

In [177]:
thongtin_khac=[]
for item in tt_khac.select('li'):
#     print(item.text)
    thongtin_khac.append(item.text.strip())
dict_detail['Thông tin khác'] = thongtin_khac
# dict_detail['Thông tin khác']  

In [185]:
# dict_detail

## Job tags/ Skills

In [178]:
jobtags_skills=[]
tag_skills = soup_level1.select('div[class="job-tags"] > ul > li')
for tag in tag_skills:
    jobtags_skills.append(tag.text)
dict_detail['Job Tags/Skills'] = jobtags_skills
# dict_detail['Job Tags/Skills']

# Thông tin công ty

In [76]:
company_info = soup_level1.select('section[class="company-overview"]')
company_info

[]

[]